# Submit Jobrun

## 1. Input Data

In [1]:
import requests
import json
from io import BytesIO
import pandas as pd
from zipfile import ZipFile

job_data = pd.read_csv(r"C:\_Working\PAT_20201019\debug\test_jobrun_input2.csv") #data/test_jobrun.csv')
job_data

,PseudoPolicyID,PolicyID,Limit,Participation,Retention,PolPrem,Stack,TIV,RatingGroup
0,146975_4418990,146975,500000000,1.0,59462.42,513499.00,4418990,5467663.0,14.0
1,150448_4349144,150448,200000000,1.0,23782.53,369032.00,4349144,21565393.0,14.0
2,157768_4404398,157768,295895093,1.0,50273.97,253952.00,4404398,2856727.0,14.0
3,143094_4443112,143094,128762896,1.0,7890.00,96908.00,4443112,7890.0,14.0
4,140586_4423882,140586,13500000,1.0,14041.10,47985.00,4423882,3100000.0,39.0
...,...,...,...,...,...,...,...,...,...
188356,161645_4342004,161645,340554825,1.0,100000.00,1959921.78,4342004,955000.0,11.0
188357,143271_4387082,143271,117507191,1.0,17491.37,162241.00,4387082,34650321.0,11.0
188358,143219_4319949,143219,250000000,1.0,51804.44,231663.00,4319949,1515284.0,11.0
188359,156824_4404656,156824,100000000,1.0,502997.93,806999.00,4404656,24424723.0,39.0


## 2. Input Parameters

In [2]:
job_para = json.load(open('data/test_jobrun.json'))
print(json.dumps(job_para,indent=2))

{
  "data_source_type": "User_Upload",
  "valid_rules": 1073741826,
  "type_of_rating": "PSOLD",
  "curve_id": 2016,
  "psold": {
    "curve_persp": "Gross",
    "peril_subline": "All_Perils",
    "trend_factor": 1.035,
    "average_accident_date": "1/1/2022",
    "blending_type": "missing_invalid",
    "blending_weights": [
      1,
      1,
      1,
      1,
      1,
      1,
      1,
      1,
      1,
      1,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0
    ],
    "hpr_blending": false
  },
  "loss_alae_ratio": 1,
  "additional_coverage": 2,
  "deductible_treatment": "Retains_Limit",
  "coverage_type": "Building_Contents_BI",
  "user_name": "Chengyou Xiao",
  "user_email": "chengyou.xiao@guycarp.com"
}


## 3. Request to the service

In [3]:
files = {
    'para': json.dumps(job_para),
    'data': job_data.to_csv(index=False)
}
resp = requests.post("http://frkcat-ws10:8002/api/job?jobrun=true", files=files)
if resp.status_code == 200:
    with ZipFile(BytesIO(resp.content),'r') as zf:
        df_res =  pd.read_csv(zf.open('pat_res.csv'))

df_res
# .sort_values('PseudoPolicyID')

# 20K records: 6s
# 188K: 28s
# 1m35s vs 26.3s

,PseudoPolicyID,PolicyID,Limit,Participation,Retention,PolPrem,Stack,TIV,RatingGroup,LossRatio,AOIr,PolLAS,DedLAS,Premium
0,146975_4418990,146975,500000000,1.0,59462.42,513499.0,4418990,5467663.0,14.0,1.0,35,36704.035183,12485.993205,4539.757777
1,146975_4418920,146975,500000000,1.0,59462.42,513499.0,4418920,9491838.0,14.0,1.0,38,42363.770028,12938.135379,5515.939475
2,146975_4418969,146975,500000000,1.0,34462.42,513499.0,4418969,2681759.0,14.0,1.0,32,33460.310525,9856.339425,4424.648015
3,146975_4418922,146975,500000000,1.0,59462.42,513499.0,4418922,3773140.0,14.0,1.0,34,35143.342775,12375.398255,4267.931871
4,146975_4418973,146975,500000000,1.0,59462.42,513499.0,4418973,28927571.0,14.0,1.0,42,49447.671868,13264.411960,6782.680273
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188356,147056_4369206,147056,142013,1.0,1385.22,12890.0,4369206,142013.0,8.0,1.0,19,10679.863801,1193.392397,12890.000000
188357,141206_4369315,141206,3333,1.0,1500.00,1826.0,4369315,3333.0,8.0,1.0,1,2284.286494,1124.949936,1826.000000
188358,149673_4297220,149673,16199,1.0,1030.68,1729.0,4297220,16199.0,30.0,1.0,10,5618.598340,931.061118,1729.000000
188359,142487_4452765,142487,2262615,1.0,3869.86,7859.0,4452765,2262615.0,34.0,1.0,31,44874.293490,3015.303113,7859.000000


In [4]:
resp.status_code, resp.headers, df_res.shape


(200,
 {'date': 'Fri, 28 Jan 2022 15:54:38 GMT', 'server': 'uvicorn', 'content-disposition': 'attachment;filename=pat_res.zip', 'access-control-expose-headers': 'Content-Disposition', 'content-type': 'application/zip', 'content-encoding': 'gzip', 'vary': 'Accept-Encoding', 'Transfer-Encoding': 'chunked'},
 (188361, 14))